In [0]:
%python
# Fraud Monitoring & Alert System
# Monitor frauds and alerts to run daily 

from pyspark.sql.functions import col, current_timestamp, split, when, size, expr, current_date
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, BooleanType

# Retrieve data from Gold tables
gold_claims = spark.table("leomar.3gold.claims_analytics")
fraud_alerts = spark.table("leomar.3gold.realtime_fraud_detection_alerts")
provider_performance = spark.table("leomar.3gold.performance")

# Display current day's fraud alerts
current_days_fraud = spark.sql("""
SELECT * FROM leomar.3gold.realtime_fraud_detection_alerts 
ORDER BY alert_severity DESC, alert_timestamp DESC
""")

#display(current_days_fraud)

# Fraud Trends Analysis
fraud_trends_analysis = spark.sql("""
SELECT 
  processing_month,
  COUNT(*) as total_claims,
  SUM(CASE WHEN fraud_risk_score > 0.7 THEN 1 ELSE 0 END) as high_risk_claims,
  ROUND(AVG(fraud_risk_score), 3) as avg_fraud_score,
  ROUND(SUM(CASE WHEN fraud_risk_score > 0.7 THEN claim_amount ELSE 0 END), 2) as high_risk_amount
FROM leomar.3gold.claims_analytics
WHERE processing_month >= date_format(add_months(current_date(), -6), 'yyyy-MM')
GROUP BY processing_month
ORDER BY processing_month
""")

print(f"Fraud Trends Analysis\n")
display(fraud_trends_analysis)

# Provider Compliance Reporting

provider_compliance_report = spark.sql("""
SELECT 
  provider_id,
  provider_name,
  tin,
  total_claims,
  total_amount,
  avg_claim_amount,
  avg_fraud_score,
  high_risk_claims,
  ROUND((high_risk_claims / total_claims) * 100, 2) as high_risk_percentage,
  CASE 
    WHEN (high_risk_claims / total_claims) > 0.3 THEN 'REVIEW REQUIRED'
    WHEN (high_risk_claims / total_claims) > 0.1 THEN 'MONITOR'
    ELSE 'COMPLIANT'
  END as compliance_status
FROM leomar.3gold.provider_performance
WHERE reporting_period = date_format(current_date(), 'yyyy-MM')
ORDER BY high_risk_percentage DESC
""")

print(f"Provider Compliance Reporting\n")
display(provider_compliance_report)

# Member Risk Profile

member_risk_profiles = spark.sql("""
SELECT 
  member_id,
  first_name,
  last_name,
  claims_count,
  total_claimed,
  member_risk_score,
  CASE 
    WHEN member_risk_score > 0.8 THEN 'HIGH RISK'
    WHEN member_risk_score > 0.5 THEN 'MEDIUM RISK'
    ELSE 'LOW RISK'
  END as risk_category
FROM leomar.3gold.member_claims_summary
--WHERE summary_period = date_format(current_date(), 'yyyy-MM-dd')
ORDER BY member_risk_score DESC
LIMIT 100
""")

print(f"Member Risk Profile\n")
display(member_risk_profiles)

# Critical Alerts Detection & Notification
# To find critical alerts that need immediate attention

# Applying spiltting the member name
critical_alerts_query = spark.sql("""
SELECT 
  a.claim_id,
  a.member_id,
  a.provider_id,
  a.claim_amount,
  a.diagnosis_code,
  a.alert_severity,
  a.alert_reason,
  a.alert_timestamp,
  split(m.member_name, ' ')[0] as first_name,
  array_join(slice(split(m.member_name, ' '), 2, size(split(m.member_name, ' ')) - 1), ' ') as last_name, 
  p.provider_name
FROM leomar.3gold.realtime_fraud_detection_alerts a
LEFT JOIN leomar.2silver.members m ON a.member_id = m.member_id
LEFT JOIN leomar.2silver.providers p ON a.provider_id = p.provider_id
WHERE a.alert_severity = 'Critical'
  AND date(a.alert_timestamp) = current_date()
ORDER BY a.claim_amount DESC
""")

print(f"Critical Alerts\n")
display(critical_alerts_query)

# Email Alert Function
def send_fraud_alert_email(alert_data):
    """
    Send email alert for critical fraud cases
    """
    critical_count = alert_data.count()
    
    if critical_count > 0:
        # Prepare email content
        subject = f"🚨 Fraud Alert: {critical_count} Critical Cases Detected"
        
        message = f"""
        Critical Fraud Alerts - {date.today()}
        Number of critical alerts: {critical_count}
        
        Top 5 Critical Cases:
        """
        
        # Add top 5 cases to email
        for row in alert_data.limit(5).collect():
            # Handle potential None values
            first_name = row.first_name if row.first_name else "Unknown"
            last_name = row.last_name if row.last_name else ""
            provider_name = row.provider_name if row.provider_name else "Unknown Provider"
            claim_amount = float(row.claim_amount) if row.claim_amount else 0.0
            alert_reason = row.alert_reason if row.alert_reason else "Unknown"
            
            message += f"""
            Claim ID: {row.claim_id}
            Member: {first_name} {last_name}
            Provider: {provider_name}
            Amount: ${claim_amount:,.2f}
            Reason: {alert_reason}
            ------------------------------
            """
        
        # Send email (configure your email settings)
        print(f"Would send email with subject: {subject}")
        print(f"To: lmlosarito@hotmail.com")
        print(f"Body: {message}")
        
        # configure email here
        dbutils.notify.send(
            subject=subject,
            body=message,
            recipients=["lmlosarito@hotmail.com"]
        )
        
        return True
    else:
        print("No critical alerts to report today.")
        return False

# Send email alerts
email_sent = send_fraud_alert_email(critical_alerts_query)

# Save Compliance Reports for auditing
(provider_compliance_report
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("leomar.audit.provider_compliance_daily"))

(member_risk_profiles
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("leomar.audit.member_risk_daily"))

# Check data quality metrics
data_quality_check = spark.sql("""
SELECT 
  current_timestamp() as check_timestamp,
  (SELECT COUNT(*) FROM leomar.3gold.claims_analytics) as total_claims,
  (SELECT COUNT(*) FROM leomar.3gold.realtime_fraud_detection_alerts 
   WHERE date(alert_timestamp) = current_date()) as today_alerts,
  (SELECT ROUND(AVG(fraud_risk_score), 3) 
   FROM leomar.3gold.claims_analytics) as overall_fraud_score
""")

display(data_quality_check)

# Log Monitoring Results
schema = StructType([
    StructField("check_timestamp", TimestampType(), False),
    StructField("critical_alerts", IntegerType(), False),
    StructField("providers_needing_review", IntegerType(), False),
    StructField("email_sent", BooleanType(), False)
])

monitoring_log = spark.createDataFrame([(
    current_timestamp(),
    critical_alerts_query.count(),
    provider_compliance_report.filter(col("compliance_status") != "COMPLIANT").count(),
    email_sent
)], schema)

(monitoring_log
 .write
 .format("delta")
 .mode("append")
 .option("overwriteSchema", "true")
 .saveAsTable("leomar.audit.fraud_monitoring_log"))

# Print summary
print("="*60)
print("FRAUD MONITORING SUMMARY")
print("="*60)
print(f"Execution Time: {datetime.now()}")
print(f"Critical Alerts Found: {critical_alerts_query.count()}")
print(f"Providers Needing Review: {provider_compliance_report.filter(col('compliance_status') != 'COMPLIANT').count()}")
print(f"Email Alert Sent: {'Yes' if email_sent else 'No'}")
print("="*60)
